In [ ]:
import pandas as pd

# Load the data file
df = pd.read_csv(r"C:\Users\nicol\OneDrive\DAT490\RQ2_Crime_Final_Pct.csv")
df.head()

In [ ]:
# Find columns that have only 1 unique value per tract (ACS variables)
tract_level_cols = []

for col in df.columns:
    if col in ["Time", "Highest Offense Description", "Crime_Category", "Census Block Group"]:
        continue
    
    sample = df.groupby("tract_geoid")[col].nunique().max()
    
    if sample == 1:
        tract_level_cols.append(col)

tract_level_cols

In [ ]:
# List of all tract-level ACS columns
acs_cols = [
 'white_alone_pct',
 'white_non_hispanic_pct',
 'black_african_american_pct',
 'asian_alone_pct',
 'hispanic_latino_pct',
 'poverty_families_pct',
 'poverty_all_people_pct',
 'unemployment_pct',
 'unemployment_female_pct',
 'employment_pct',
 'bachelors_degree_plus_pct',
 'graduate_degree_pct',
 'high_school_graduate_pct',
 'some_college_pct',
 'english_only_pct',
 'non_english_home_pct',
 'disability_pct',
 'veteran_pct',
 'owner_occupied_units_count',
 'owner_occupied_pct',
 'renter_occupied_units_count',
 'renter_occupied_pct',
 'total_housing_units',
 'occupied_housing_units',
 'vacant_housing_units',
 'vacant_housing_pct',
 'avg_household_size_owner',
 'avg_household_size_renter',
 'avg_household_size',
 'avg_family_size',
 'total_households',
 'median_home_value',
 'median_gross_rent'
]

# Build the full tract-year aggregated dataset
agg_dict = {"incidents": ("year_occurred", "size")}

for col in acs_cols:
    agg_dict[col] = (col, "first")

tract_year_df = (
    df
    .groupby(["tract_geoid", "year_occurred"])
    .agg(**agg_dict)
    .reset_index()
)

tract_year_df.head()

In [ ]:
# Remove rows with missing household counts
clean_df = tract_year_df.dropna(subset=["total_households"]).copy()

# Remove rows with zero households
clean_df = clean_df[clean_df["total_households"] > 0].copy()

clean_df.head(), clean_df.shape

In [ ]:
clean_df["crime_rate_per_1000_hh"] = (
    clean_df["incidents"] / clean_df["total_households"] * 1000
)

clean_df[["tract_geoid", "year_occurred", "incidents", "total_households", "crime_rate_per_1000_hh"]].head()

In [ ]:
# Step 1: get the model dataframe
acs_vars = [
    'white_alone_pct', 'white_non_hispanic_pct', 'black_african_american_pct',
    'asian_alone_pct', 'hispanic_latino_pct', 'poverty_families_pct',
    'poverty_all_people_pct', 'unemployment_pct', 'unemployment_female_pct',
    'employment_pct', 'bachelors_degree_plus_pct', 'graduate_degree_pct',
    'high_school_graduate_pct', 'some_college_pct', 'english_only_pct',
    'non_english_home_pct', 'disability_pct', 'veteran_pct',
    'owner_occupied_units_count', 'owner_occupied_pct',
    'renter_occupied_units_count', 'renter_occupied_pct',
    'total_housing_units', 'occupied_housing_units', 'vacant_housing_units',
    'vacant_housing_pct', 'avg_household_size_owner',
    'avg_household_size_renter', 'avg_household_size', 'avg_family_size',
    'total_households', 'median_home_value', 'median_gross_rent'
]

model_df = clean_df[['tract_geoid', 'year_occurred', 'crime_rate_per_1000_hh'] + acs_vars].copy()

model_df.head()

In [ ]:
import statsmodels.api as sm

# Step 2: Pick predictor
X = model_df[['poverty_all_people_pct']]
y = model_df['crime_rate_per_1000_hh']

# Step 3: Add constant
X = sm.add_constant(X)

# Step 4: Fit model
poverty_model = sm.OLS(y, X).fit()

poverty_model.summary()

In [ ]:
X = model_df[['renter_occupied_pct']]
y = model_df['crime_rate_per_1000_hh']

X = sm.add_constant(X)
rent_model = sm.OLS(y, X).fit()
rent_model.summary()

In [ ]:
X = model_df[['bachelors_degree_plus_pct']]
y = model_df['crime_rate_per_1000_hh']

X = sm.add_constant(X)
edu_model = sm.OLS(y, X).fit()
edu_model.summary()

In [ ]:
X = model_df[['unemployment_pct']]
y = model_df['crime_rate_per_1000_hh']

X = sm.add_constant(X)
unemp_model = sm.OLS(y, X).fit()
unemp_model.summary()

In [ ]:
X = model_df[['hispanic_latino_pct']]
y = model_df['crime_rate_per_1000_hh']

X = sm.add_constant(X)
hisp_model = sm.OLS(y, X).fit()
hisp_model.summary()

In [ ]:
X = model_df[['vacant_housing_pct']]
y = model_df['crime_rate_per_1000_hh']

X = sm.add_constant(X)
vac_model = sm.OLS(y, X).fit()
vac_model.summary()

In [ ]:
X = model_df[['owner_occupied_pct']]
y = model_df['crime_rate_per_1000_hh']

X = sm.add_constant(X)
own_model = sm.OLS(y, X).fit()
own_model.summary()

In [ ]:
model_df['median_home_value_num'] = pd.to_numeric(
    model_df['median_home_value'],
    errors='coerce'   # turns non-numeric stuff into NaN instead of crashing
)

# Quick check:
model_df[['median_home_value', 'median_home_value_num']].head(10)

In [ ]:
import statsmodels.api as sm

mask = model_df['median_home_value_num'].notna()
temp = model_df.loc[mask].copy()

X = temp[['median_home_value_num']]
y = temp['crime_rate_per_1000_hh']

X = sm.add_constant(X)
value_model = sm.OLS(y, X).fit()
value_model.summary()

In [ ]:
models_info = [
    ("poverty_all_people_pct", poverty_model, "poverty_all_people_pct"),
    ("renter_occupied_pct",    rent_model,    "renter_occupied_pct"),
    ("owner_occupied_pct",     own_model,     "owner_occupied_pct"),
    ("vacant_housing_pct",     vac_model,     "vacant_housing_pct"),
    ("bachelors_degree_plus_pct", edu_model,  "bachelors_degree_plus_pct"),
    ("median_home_value_num",  value_model,   "median_home_value_num"),
    ("hispanic_latino_pct",    hisp_model,    "hispanic_latino_pct"),
    ("unemployment_pct",       unemp_model,   "unemployment_pct"),
]

rows = []
for pred_name, model, var_key in models_info:
    coef = model.params[var_key]
    pval = model.pvalues[var_key]
    r2   = model.rsquared
    
    rows.append({
        "predictor": pred_name,
        "coef": coef,
        "p_value": pval,
        "r_squared": r2
    })

model_comp_df = pd.DataFrame(rows)

# Sort from highest to lowest R²
model_comp_df = model_comp_df.sort_values("r_squared", ascending=False).reset_index(drop=True)

model_comp_df.style.format({
    "coef": "{:.4f}",
    "p_value": "{:.3g}",
    "r_squared": "{:.3f}"
}).hide(axis="index")
model_comp_df

In [ ]:
import matplotlib.pyplot as plt

plot_df = model_comp_df.copy()

plot_df = plot_df.sort_values("r_squared", ascending=True)  # ascending 

plt.figure(figsize=(8, 5))

plt.barh(plot_df["predictor"], plot_df["r_squared"])

plt.xlabel("R-squared")
plt.ylabel("Predictor")
plt.title("Single-Variable Models: Variance Explained in Crime Rate")

plt.grid(axis="x", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Make a sorted copy: weakest at bottom, strongest at top
plot_df = model_comp_df.copy().sort_values("r_squared", ascending=True)

y_pos = range(len(plot_df))

plt.figure(figsize=(8, 5))

plt.hlines(y=y_pos,
           xmin=0,
           xmax=plot_df["r_squared"],
           linewidth=1.5)

plt.plot(plot_df["r_squared"], y_pos, "o")

# Y-axis labels: predictor names
plt.yticks(ticks=y_pos, labels=plot_df["predictor"])

# Axis labels
plt.xlabel("R-squared (single-variable OLS model)")
plt.ylabel("Predictor")
plt.title("Explanatory Power of Single-Predictor Models for Crime Rate")

for i, row in plot_df.iterrows():
    r2 = row["r_squared"]
    coef = row["coef"]
    y = list(plot_df.index).index(i)
    plt.text(r2 + 0.003, y, f"β = {coef:.2f}", va="center", fontsize=8)

plt.grid(axis="x", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()